In [0]:
import pyspark
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
dbutils.secrets.listScopes()

[SecretScope(name='healthscope')]

In [0]:
dbutils.secrets.list(scope = 'healthscope')

[SecretMetadata(key='healthdata')]

In [0]:
spark.conf.set("fs.azure.account.key.healthdata.dfs.core.windows.net",
    dbutils.secrets.get(scope="healthscope", key="healthdata"))

In [0]:
dbutils.fs.ls("abfss://rawdata@healthdata.dfs.core.windows.net")

[FileInfo(path='abfss://rawdata@healthdata.dfs.core.windows.net/Patient_records.csv', name='Patient_records.csv', size=5110, modificationTime=1710596297000),
 FileInfo(path='abfss://rawdata@healthdata.dfs.core.windows.net/claims.csv', name='claims.csv', size=11438, modificationTime=1710615336000),
 FileInfo(path='abfss://rawdata@healthdata.dfs.core.windows.net/disease.csv', name='disease.csv', size=1489, modificationTime=1710596297000),
 FileInfo(path='abfss://rawdata@healthdata.dfs.core.windows.net/group.csv', name='group.csv', size=4390, modificationTime=1710596297000),
 FileInfo(path='abfss://rawdata@healthdata.dfs.core.windows.net/hospital.csv', name='hospital.csv', size=1328, modificationTime=1710596297000),
 FileInfo(path='abfss://rawdata@healthdata.dfs.core.windows.net/subgroup.csv', name='subgroup.csv', size=561, modificationTime=1710596297000),
 FileInfo(path='abfss://rawdata@healthdata.dfs.core.windows.net/subscriber.csv', name='subscriber.csv', size=12061, modificationTime=1

In [0]:
patient_recor_data = spark.read.csv("abfss://rawdata@healthdata.dfs.core.windows.net/Patient_records.csv", header = True, escape = '"')

In [0]:
#patient_recor_data.show(5)

In [0]:
#patient name null value, also check for duplicates

In [0]:
#patient_recor_data.select([count(when(isnan(c) | col(c).isNull(),c)).alias(c) for c in patient_recor_data.columns]).show()

In [0]:
#patient_recor_data.groupBy(["Patient_id","Patient_name","patient_gender","patient_birth_date","patient_phone", "disease_name","city","hospital_id"]).count().where("count > 1").show()

In [0]:
patient_recor_data = patient_recor_data.dropDuplicates()

In [0]:
# fill the missing value
patient_recor_data = patient_recor_data.fillna({"Patient_name":"Guest/NA","patient_phone": "NA"})


In [0]:
#writing clean dataframe into stagging
#boilerplate code(standard code for ceratin operation)
stagging_path = "abfss://stagedata@healthdata.dfs.core.windows.net"
patient_recor_data.coalesce(1).write.mode("append").option("header","true").format("com.databricks.spark.csv").save(stagging_path)
files = dbutils.fs.ls(stagging_path)
output_file = [x for x in files if x.name.startswith("part-")] #to read only part files
dbutils.fs.mv(output_file[0].path,"%s/patiend_recordstagging.csv" % stagging_path)
files_to_delete = [file.path for file in files if file.name.startswith("_")]
for file_path in files_to_delete:
                  dbutils.fs.rm(file_path, True)